In [1]:
import random
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

In [2]:
mp_df = pd.read_pickle('manpages-db-clean.pkl')
mp_df = mp_df.head(75)
for i in range(len(mp_df['command'])):
    mp_df['command'][i] = mp_df['command'][i].lower()
print(mp_df)

              command                                            options  \
0            0desktop                                                NaN   
1            0install  <pre>\n       The first non-option argument to...   
2             0launch  <pre>\n       0launch takes the same options a...   
3   0store-secure-add                                                NaN   
4              0store  <pre>\n       <b>-h</b>, <b>--help</b>\n      ...   
..                ...                                                ...   
70            aa-exec                                     <b>aa-exec</b>   
71                 aa  <pre>\n       <b>aa</b> does not accept any op...   
72               aa3d  <pre>\n       <b>-w</b> width\n              T...   
73            aac-enc          <pre>\n       See program binary.\n</pre>   
74             aafire                                                NaN   

                                                flags  
0                              

In [3]:
def string_to_list(word):
    return [char for char in word]

In [4]:
def list_to_string(in_list):
    word = ''
    for char in in_list:
        word = word+(char)
    return word

In [5]:
def get_nearby_keys(key):
    nearby_keys = {
        'q': ['w', 'a', 's'],
        'w': ['q', 'e', 'a', 's'],
        'e': ['w', 'r', 's', 'd'],
        'r': ['e', 't', 'd', 'f'],
        't': ['r', 'y', 'f', 'g'],
        'y': ['t', 'u', 'g', 'h'],
        'u': ['y', 'i', 'h', 'j'],
        'i': ['u', 'o', 'j', 'k'],
        'o': ['i', 'p', 'k', 'l'],
        'p': ['o', 'l'],
        'a': ['q', 'w', 's', 'z'],
        's': ['w', 'e', 'd', 'x', 'z', 'a'],
        'd': ['s', 'e', 'r', 'f', 'c', 'x'],
        'f': ['r', 't', 'g', 'v', 'c', 'd'],
        'g': ['t', 'y', 'h', 'b', 'v', 'f'],
        'h': ['y', 'u', 'j', 'n', 'b', 'g'],
        'j': ['u', 'i', 'k', 'm', 'n', 'h'],
        'k': ['i', 'o', 'l', 'm', 'j'],
        'l': ['k', 'o', 'p'],
        'z': ['a', 's', 'x'],
        'x': ['z', 's', 'd', 'c'],
        'c': ['x', 'd', 'f', 'v'],
        'v': ['c', 'f', 'g', 'b'],
        'b': ['v', 'g', 'h', 'n'],
        'n': ['b', 'h', 'j', 'm'],
        'm': ['n', 'j', 'k']
    }
    if key in nearby_keys.keys():
        return nearby_keys[key]
    else:
        return [key]

In [6]:
def char_to_num(in_char):
    chars = {
        'a': 1, 
        'b': 2,
        'c': 3,
        'd': 4,
        'e': 5,
        'f': 6,
        'g': 7,
        'h': 8,
        'i': 9,
        'j': 10,
        'k': 11,
        'l': 12,
        'm': 13,
        'n': 14,
        'o': 15,
        'p': 16,
        'q': 17,
        'r': 18,
        's': 19,
        't': 20,
        'u': 21,
        'v': 22,
        'w': 23,
        'x': 24,
        'y': 25,
        'z': 26
    }
    if in_char in chars:
        return chars[in_char]
    else:
        return 0

In [7]:
# def swap_letters(word):
#     typo_arr = []
#     word_as_arr = string_to_list(word)
#     for a in range(len(word_as_arr)):
#         temp1 = word_as_arr[a]
#         nearby_letters1 = get_nearby_keys(temp1)
#         for letter in nearby_letters1:
#             word_as_arr[a] = letter
#             typo_arr.append(list_to_string(word_as_arr))
#             for b in range(a+1, len(word_as_arr)):
#                     temp2 = word_as_arr[b]
#                     nearby_letters2 = get_nearby_keys(temp2)
#                     for letter2 in nearby_letters2:
#                         word_as_arr[b] = letter2
#                         typo_arr.append(list_to_string(word_as_arr))
#                         word_as_arr[b] = temp2
#             word_as_arr[a] = temp1
#     return typo_arr

In [8]:
def swap_letters(word):
    typo_arr = []
    word_as_arr = string_to_list(word)
    for a in range(len(word_as_arr)):
        temp1 = word_as_arr[a]
        mistake_letter1 = random.choice(get_nearby_keys(temp1))
        word_as_arr[a] = mistake_letter1
        typo_arr.append(list_to_string(word_as_arr))
        try:
            b = random.choice([*range(a, len(word_as_arr))])
            temp2 = word_as_arr[b]
            mistake_letter2 = random.choice(get_nearby_keys(temp2))
            word_as_arr[b] = mistake_letter2
            typo_arr.append(list_to_string(word_as_arr))
            word_as_arr[b] = temp2
        except:
            #do nothing
            continue
        word_as_arr[a] = temp1
    return typo_arr

In [9]:
x = (swap_letters('command'))
duplicates_found = False
for i in range(len(x)):
    for j in range(i+1, len(x)):
        if x[i] == x[j]:
            duplicates_found = True
            print("DUPLICATE:")
            print(str(x[i])+" at "+str(i))
            print(str(x[j])+" at "+str(j))
            print()
print(duplicates_found)
print(x)

False
['fommand', 'tommand', 'cpmmand', 'cpmmajd', 'cokmand', 'cokkand', 'comnand', 'comnahd', 'commwnd', 'commwne', 'commabd', 'commabe', 'commanr', 'commanf']


In [10]:
length = len(mp_df['command'])
typo_set = [[],[],[]] # col 0 is typo, col 1 is flags, col 2 is correct command

def add_typos(original_command, i):
    mistyped_commands = swap_letters(original_command)
    flags = mp_df['flags'][i]
    flags.append('') # add a "no flags" option
    for j in range(len(mistyped_commands)):
        for flag in flags:
            typo_set[0].append(mistyped_commands[j])
            typo_set[1].append(flag)
            typo_set[2].append(original_command)
    return typo_set

typo_set = Parallel(n_jobs = -1)(delayed(add_typos)(mp_df['command'][i], i) for i in tqdm(range(length)))
print(len(typo_set))

100%|███████████████████████████████████████████| 75/75 [00:00<00:00, 93.94it/s]

75


In [11]:
typo_df = pd.DataFrame()
for i in tqdm(range(len(typo_set))):
    sub_dataframe = pd.DataFrame()
    sub_dataframe['typo'] = (typo_set[i][0])
    sub_dataframe['flag'] = typo_set[i][1]
    sub_dataframe['command'] = typo_set[i][2]
    typo_df = typo_df.append(sub_dataframe, ignore_index = True)
print(typo_df)

100%|██████████████████████████████████████████| 75/75 [00:00<00:00, 422.96it/s]

          typo flag   command
0     0desktop       0desktop
1     0dewktop       0desktop
2     0resktop       0desktop
3     0resltop       0desktop
4     0dssktop       0desktop
...        ...  ...       ...
9969    aafjre         aafire
9970    aafide         aafire
9971    aafidd         aafire
9972    aafird         aafire
9973    aafirx         aafire

[9974 rows x 3 columns]


In [12]:
def get_longest():
    length = 0
    longest_entry = ''
    for entry in typo_df['command']:
        if len(entry) > length:
            length = len(entry)
            longest_entry = entry
    return length, longest_entry
    
highest_len, longest_entry = get_longest()
print(highest_len)
print(longest_entry)

30
a11y-profile-manager-indicator


In [13]:
def encode_typos(in_arr, highest_len):
    out_list = []
    for i in tqdm(range(len(in_arr))):
        s = string_to_list(in_arr[i])
        for j in range(len(s)):
            s[j] = char_to_num(s[j])
        for k in range(highest_len-len(s)):
            s.append(0)
        out_list.append(s)
    return out_list
encoded_typos = encode_typos(typo_df['typo'], highest_len)
typo_df['encoded_typo'] = encoded_typos

100%|████████████████████████████████████| 9974/9974 [00:00<00:00, 93029.54it/s]


In [14]:
def get_flat_set(in_arr):
    out_arr = []
    for element in in_arr:
        if not(element in out_arr):
            out_arr.append(element)
    return out_arr

def encode_strings(in_arr):
    encoded_flags = []
    flat_set = get_flat_set(in_arr)
    for element in tqdm(in_arr):
        encoded_flags.append(flat_set.index(element))
    return encoded_flags

In [15]:
encoded_flags = encode_strings(typo_df['flag'])
typo_df['encoded_flag'] = encoded_flags

100%|███████████████████████████████████| 9974/9974 [00:00<00:00, 562867.32it/s]


In [16]:
encoded_commands = encode_strings(typo_df['command'])
typo_df['encoded_command'] = encoded_commands

100%|███████████████████████████████████| 9974/9974 [00:00<00:00, 984352.29it/s]


In [17]:
combined_input_list = []
for x in tqdm(range(len(encoded_flags))):
    combined_input_list.append(encoded_typos[x]+[encoded_flags[x]])
typo_df['encoded_input'] = combined_input_list

100%|███████████████████████████████████| 9974/9974 [00:00<00:00, 707300.38it/s]


In [18]:
num_unique_commands = typo_df['encoded_command'].nunique()
print(num_unique_commands)

74


In [19]:
one_hot_arr = []
for c in tqdm(typo_df['encoded_command']):
    new_one_hot = [0]*num_unique_commands
    new_one_hot[c] = 1
    one_hot_arr.append(new_one_hot)
typo_df['one_hot_encoded_command'] = one_hot_arr

100%|███████████████████████████████████| 9974/9974 [00:00<00:00, 105754.08it/s]


In [20]:
print(typo_df[typo_df['flag']!=''])
print(typo_df['encoded_flag'])

          typo       flag   command  \
16    0install         -c  0install   
17    0install  --console  0install   
18    0install  --dry-run  0install   
19    0install         -h  0install   
20    0install     --help  0install   
...        ...        ...       ...   
9942      aa3d         -s      aa3d   
9943      aa3d         -t      aa3d   
9944      aa3d         -r      aa3d   
9945      aa3d         -f      aa3d   
9946      aa3d         -d      aa3d   

                                           encoded_typo  encoded_flag  \
16    [0, 9, 14, 19, 20, 1, 12, 12, 0, 0, 0, 0, 0, 0...             1   
17    [0, 9, 14, 19, 20, 1, 12, 12, 0, 0, 0, 0, 0, 0...             2   
18    [0, 9, 14, 19, 20, 1, 12, 12, 0, 0, 0, 0, 0, 0...             3   
19    [0, 9, 14, 19, 20, 1, 12, 12, 0, 0, 0, 0, 0, 0...             4   
20    [0, 9, 14, 19, 20, 1, 12, 12, 0, 0, 0, 0, 0, 0...             5   
...                                                 ...           ...   
9942  [1, 1, 0, 4, 0

In [21]:
typo_df.to_pickle('typo_df.pkl')